In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import cv2 as cv
import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings("ignore")

In [ ]:
train_ds = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/train.csv')
test_ds = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/test.csv')
sample_submission = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/sample_submission.csv')

train_path = '/kaggle/input/petfinder-pawpularity-score/train/'
test_path = '/kaggle/input/petfinder-pawpularity-score/test/'

In [ ]:
train_ds.head()

In [ ]:
train_features = [col for col in train_ds.columns if col not in ['Id', 'Pawpularity']]
test_features = [col for col in test_ds.columns if col != 'Id']

In [ ]:
train_features == test_features

In [ ]:
X_train = train_ds[train_features]
X_test = test_ds[test_features]

y_train = train_ds['Pawpularity']

In [ ]:
try:
    del int_X_train, int_X_test, int_y_train
except:
    pass

int_X_train = X_train.values
int_X_test = X_test.values
int_y_train = y_train.values

In [ ]:
inputs = keras.Input(shape=(12,))

x = layers.Dense(32, activation='relu')(inputs)
block_one = layers.LayerNormalization()(x)

x = layers.Dense(32, activation='relu')(block_one)
x = layers.Dense(64, activation='relu')(x)
x = layers.LayerNormalization()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(32, activation='relu')(x)
x = layers.LayerNormalization()(x)
block_two = layers.Add()([x, block_one])

x = layers.Dense(32, activation='relu')(block_one)
x = layers.Dense(64, activation='relu')(x)
x = layers.LayerNormalization()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(32, activation='relu')(x)
x = layers.LayerNormalization()(x)
block_three = layers.Add()([x, block_one])

final_block = layers.Add()([block_two, block_three])

x = layers.Dense(32, activation='relu')(final_block)
x = layers.LayerNormalization()(x)
x = layers.Dense(16, activation='relu')(x)
x = layers.Dense(8, activation='relu')(x)

outputs = layers.Dense(1)(x)

model = keras.Model(inputs, outputs)
model.summary()

In [ ]:
model.compile(optimizer='rmsprop',
             loss='mae',
             metrics=['mse'])

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint('petfinder.keras', save_best_only=True)
]

history = model.fit(int_X_train, int_y_train, epochs=30, validation_split=0.2, shuffle=True, callbacks=callbacks)

In [ ]:
model = keras.models.load_model('petfinder.keras')
model.predict(int_X_test)

In [ ]:
hist = history.history

In [ ]:
training_loss = hist['loss']
training_mse = hist['mse']
validation_loss = hist['loss']
validation_mse = hist['mse']

epochs_range = range(30)

plt.figure(figsize=(12,8))
plt.subplot(1,2,1)
plt.plot(epochs_range, training_loss, label='training_loss')
plt.plot(epochs_range, validation_loss, label='validation_loss')
plt.title('Training Vs Validation loss')
plt.legend()

plt.subplot(1,2,2)
plt.plot(epochs_range, training_mse, label='training_mse')
plt.plot(epochs_range, validation_mse, label='validation_loss')
plt.title("Training Vs Validation mse")
plt.legend()

plt.show()

In [ ]:
predictions = model.predict(int_X_test)
mean = np.mean(predictions, axis=1)

In [ ]:
submission = pd.DataFrame({'Id':sample_submission['Id'], 'Pawpularity':mean})

In [ ]:
submission

In [ ]:
submission.to_csv('submission.csv', index=False)